# Plots of RF Classifications feature importances

In [2]:
# Standard library
import warnings
import logging
from itertools import combinations
from functools import reduce

# Scientific computing
import numpy as np
import pandas as pd
from numpy import array
import scipy
import scipy.stats as ss
from scipy import interp
from scipy.stats import wilcoxon, ttest_rel

# Visualization
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns
import matplotlib.gridspec as gridspec
from PIL import Image
from PIL import Image, ImageDraw, ImageFont
from matplotlib.colors import LinearSegmentedColormap

# scikit-bio
from skbio.stats.distance import permanova

# BIOM format
import biom
from biom import load_table

# Scikit-learn
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    roc_curve, auc, RocCurveDisplay
)
from sklearn.model_selection import GroupKFold, StratifiedKFold
from sklearn.preprocessing import label_binarize


In [3]:
# Define file paths
files = {
    "skin_vs_nares": "../Data/RF_Feature_Importances/ASVs/Top_10_Features_with_ASV_IDs_skin_vs_nares_V4.csv",
    "skin_ADL_vs_H": "../Data/RF_Feature_Importances/ASVs/Top_10_Features_with_ASV_IDs_skin-ADL_vs_skin-H_V4.csv",
    "skin_ADNL_vs_H": "../Data/RF_Feature_Importances/ASVs/Top_10_Features_with_ASV_IDs_skin-ADNL_vs_skin-ADL_V4.csv",
    "skin_ADNL_vs_ADL": "../Data/RF_Feature_Importances/ASVs/Top_10_Features_with_ASV_IDs_skin-ADNL_vs_skin-ADL_V4.csv",
    "nares_AD_vs_H": "../Data/RF_Feature_Importances/ASVs/Top_10_Features_with_ASV_IDs_nares-AD_vs_nares-H_V4.csv"
}

# Read and process each file
rank_dfs = {}
for key, path in files.items():
    df = pd.read_csv(path)
    df = df.sort_values("mean_importance", ascending=False).reset_index(drop=True)
    # Remove rows where the index starts with ' g__ASV'
    df = df[~df["Genus"].str.strip().str.startswith('g___ASV-2')]
    df = df[~df["Genus"].str.strip().str.startswith('g___ASV-6')]
    df = df[~df["Genus"].str.strip().str.startswith('g___ASV-7')]

    df["rank"] = range(1, len(df) + 1)
    df = df[["Genus", "rank"]]
    rank_dfs[key] = df

# List of comparisons to include
keys_to_merge = ["skin_vs_nares", "skin_ADL_vs_H", "skin_ADNL_vs_H", "skin_ADNL_vs_ADL", "nares_AD_vs_H"]

# Merge all rank dfs cleanly
merged = reduce(
    lambda left, right: pd.merge(left, right, on="Genus", how="outer"),
    [rank_dfs[key].rename(columns={"rank": key}) for key in keys_to_merge]
)

# Fill missing values with max_rank + 1
max_rank = max([df["rank"].max() for df in rank_dfs.values()])
merged = merged.set_index("Genus").fillna(max_rank + 1)

merged

,skin_vs_nares,skin_ADL_vs_H,skin_ADNL_vs_H,skin_ADNL_vs_ADL,nares_AD_vs_H
Genus,,,,,
g__2011-GWC2-44-17_ASV-1,1706,1165,1169,1169,1250
g__AC-16_ASV-1,1803,1322,1560,1560,2106
g__Abditibacterium_ASV-1,1229,1450,2062,2062,1799
g__Abiotrophia_ASV-1,133,88,105,105,77
g__Abiotrophia_ASV-2,255,185,191,191,199
...,...,...,...,...,...
g___ASV-95,390,602,459,459,430
g___ASV-96,580,788,1847,1847,804
g___ASV-97,230,213,147,147,635


In [4]:
# Get top 10 genera from each comparison separately
top_genera = set()
for comparison in keys_to_merge:
    top10_in_col = merged.sort_values(comparison).head(10).index
    top_genera.update(top10_in_col)

# Subset merged to include only these genera
top10 = merged.loc[list(top_genera)]

# Sort again based on total summed rank (optional, for nicer display)
top10['sum'] = top10.sum(axis=1)
top10 = top10.sort_values('sum').drop(columns='sum')

# View final table
top10.index

Index(['g__Streptococcus_ASV-1', 'g__Staphylococcus_ASV-1',
       'g__Cutibacterium_ASV-1', 'g__Staphylococcus_ASV-2',
       'g__Haemophilus_D_734546_ASV-1', 'g__Veillonella_A_ASV-1',
       'g__Streptococcus_ASV-2', 'g__Cutibacterium_ASV-2',
       'g__Micrococcus_ASV-1', 'g__Massilia_ASV-1', 'g__Gemella_ASV-1',
       'g__F0422_ASV-1', 'g__Prevotella_ASV-1', 'g__Neisseria_563205_ASV-2',
       'g__Actinomyces_ASV-1', 'g__Granulicatella_ASV-1',
       'g__Haemophilus_D_735815_ASV-1', 'g__Escherichia_710834_ASV-1',
       'g__Corynebacterium_ASV-1', 'g__Dolosigranulum_ASV-1'],
      dtype='object', name='Genus')

In [5]:
# Replace specific genus names
top10 = top10.rename(index={
    'g__F0422_ASV-1': 'g__Veillonella_F0422_ASV-1',
})

top10.index.to_list()

['g__Streptococcus_ASV-1',
 'g__Staphylococcus_ASV-1',
 'g__Cutibacterium_ASV-1',
 'g__Staphylococcus_ASV-2',
 'g__Haemophilus_D_734546_ASV-1',
 'g__Veillonella_A_ASV-1',
 'g__Streptococcus_ASV-2',
 'g__Cutibacterium_ASV-2',
 'g__Micrococcus_ASV-1',
 'g__Massilia_ASV-1',
 'g__Gemella_ASV-1',
 'g__Veillonella_F0422_ASV-1',
 'g__Prevotella_ASV-1',
 'g__Neisseria_563205_ASV-2',
 'g__Actinomyces_ASV-1',
 'g__Granulicatella_ASV-1',
 'g__Haemophilus_D_735815_ASV-1',
 'g__Escherichia_710834_ASV-1',
 'g__Corynebacterium_ASV-1',
 'g__Dolosigranulum_ASV-1']

In [6]:
# Remove 'g__' prefix and '_ASV' from index names
top10.index = top10.index.str.replace('g__', '')
top10.index = top10.index.str.replace('_ASV', ' ASV')


In [7]:
print(top10.columns.tolist())


['skin_vs_nares', 'skin_ADL_vs_H', 'skin_ADNL_vs_H', 'skin_ADNL_vs_ADL', 'nares_AD_vs_H']


In [8]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from matplotlib.colors import LinearSegmentedColormap

# --- Truncate function to remove very white parts ---
def truncate_colormap(cmap, minval=0.0, maxval=0.8, n=100):
    new_cmap = LinearSegmentedColormap.from_list(
        f'trunc({cmap.name},{minval:.2f},{maxval:.2f})',
        cmap(np.linspace(minval, maxval, n))
    )
    return new_cmap

# --- Your custom colormaps (non-reversed!) ---
colormaps = {
    'skin_vs_nares': 'Greys',
    'skin_ADL_vs_H': 'Blues',
    'skin_ADNL_vs_H': 'Greens',
    'skin_ADNL_vs_ADL': 'Purples',
    'nares_AD_vs_H': 'Oranges'
}

# --- Transpose so comparisons are rows, features are columns ---
top10_t = top10.T

# --- Create the figure ---
fig, ax = plt.subplots(figsize=(12, 4))  # wide figure

# Normalize rows
normed_data = pd.DataFrame(index=top10_t.index, columns=top10_t.columns)

for comparison in top10_t.index:
    row = top10_t.loc[comparison]
    normed = (row - row.min()) / (row.max() - row.min())
    normed_data.loc[comparison] = normed

# --- Plot manually ---
for idx, comparison in enumerate(top10_t.index):
    base_cmap = plt.get_cmap(colormaps[comparison])
    cmap = truncate_colormap(base_cmap, 0.0, 0.8)  # Cut off super light top
    row_data = normed_data.loc[comparison].astype(float)
    
    for jdx, value in enumerate(row_data):
        corrected_value = 1 - value  # <-- flip! low values colorful, high values light
        color = cmap(corrected_value)
        
        rect = plt.Rectangle((jdx, idx), 1, 1, facecolor=color, edgecolor='white', linewidth=0.5)
        ax.add_patch(rect)
        
        original_val = top10_t.loc[comparison].iloc[jdx]
        ax.text(jdx + 0.5, idx + 0.5, f"{int(original_val)}",
                ha='center', va='center', color='black', fontsize=8)

# Set axis limits
ax.set_xlim(0, top10_t.shape[1])
ax.set_ylim(0, top10_t.shape[0])

# Set ticks
ax.set_xticks(np.arange(top10_t.shape[1]) + 0.5)
ax.set_yticks(np.arange(top10_t.shape[0]) + 0.5)
ax.set_xticklabels(top10_t.columns, rotation=45, ha='right')
ax.set_yticklabels(["All Skin vs. All Nares", "Skin ADL vs Skin H", "Skin ADNL vs Skin H", "Skin ADNL vs Skin ADL", "Nares AD vs Nares H"])

# Reverse y-axis
ax.invert_yaxis()

# Clean up
ax.set_xticks(np.arange(top10_t.shape[1]), minor=True)
ax.set_yticks(np.arange(top10_t.shape[0]), minor=True)
ax.grid(False)
ax.tick_params(which="minor", bottom=False, left=False)

# Title
plt.title("Top 10 Features from Each Classification Pooled", fontsize=16, pad=35, x=0.45)

# Smaller subtitle
plt.text(
    0.45, 1.25, 
    "(lower rank values correspond to higher feature importance)",
    ha='center', va='center',
    transform=ax.transAxes,
    fontsize=12
)

plt.tight_layout()
plt.savefig('../Plots/Analysis_figures/Random_Forest/combined_heatmap_horizontal_flipped.png', dpi=600, bbox_inches='tight')


In [9]:
# --- Make sure Genus is the index ---
top10 = top10.reset_index().set_index('Genus')
top10.index = top10.index.str.strip()  # Clean any leading/trailing spaces

# Define which comparison
group1 = ["skin_vs_nares"]

# --- Select top 10 genera from skin_vs_nares only ---
top10_group1 = top10.sort_values('skin_vs_nares').head(10)

# --- Plot ---
fig, ax = plt.subplots(figsize=(1, 6))  # Slightly shorter figure

sns.heatmap(
    top10_group1[group1],
    cmap="Greys_r",
    annot=True, fmt=".0f", linewidths=0.5,
    cbar=False,
    square=False,
    ax=ax
)

ax.set_title("Skin vs Nares", loc='right', pad=10)  # pad=10 brings it closer
ax.set_ylabel("")
ax.set_xlabel("Rank")
ax.set_xticks([])  # remove x-axis tick labels
# ax.set_aspect(0.85)  # still control cell thickness

plt.tight_layout()
plt.savefig('../Plots/Analysis_figures/Random_Forest/top10_genera_rankings_all-classifications_skin-vs-nares.png', dpi=600, bbox_inches='tight')





/var/folders/22/yck9vwx53w1c38tvj_c0_tz00000gn/T/ipykernel_47332/3391101392.py:29: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations.
  plt.tight_layout()


In [10]:
# --- Make sure Genus is the index ---
top10 = top10.reset_index().set_index('Genus')
top10.index = top10.index.str.strip()  # Clean any leading/trailing spaces

# Define which comparison
group2 = ["skin_ADL_vs_H"]

# --- Select top 10 genera from skin_ADL_vs_H only ---
top10_group2 = top10.sort_values('skin_ADL_vs_H').head(10)

# --- Plot ---
fig, ax = plt.subplots(figsize=(1, 6))  # Slightly shorter figure

sns.heatmap(
    top10_group2[group2],
    cmap="Blues_r",
    annot=True, fmt=".0f", linewidths=0.5,
    cbar=False,
    square=False,
    ax=ax
)

ax.set_title("Skin ADL vs H", loc='right', pad=10)  # pad=10 brings it closer
ax.set_ylabel("")
ax.set_xlabel("Rank")
ax.set_xticks([])  # remove x-axis tick labels
# ax.set_aspect(0.85)  # still control cell thickness

plt.tight_layout()
plt.savefig('../Plots/Analysis_figures/Random_Forest/top10_genera_rankings_all-classifications_skin-ADL-H.png', dpi=600, bbox_inches='tight')





/var/folders/22/yck9vwx53w1c38tvj_c0_tz00000gn/T/ipykernel_47332/890908534.py:29: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations.
  plt.tight_layout()


In [11]:
# --- Make sure Genus is the index ---
top10 = top10.reset_index().set_index('Genus')
top10.index = top10.index.str.strip()  # Clean any leading/trailing spaces

# Define which comparison
group3 = ["skin_ADNL_vs_H"]

# --- Select top 10 genera from skin_ADL_vs_H only ---
top10_group3 = top10.sort_values('skin_ADNL_vs_H').head(10)

# --- Plot ---
fig, ax = plt.subplots(figsize=(1, 6))  # Slightly shorter figure

sns.heatmap(
    top10_group3[group3],
    cmap="Greens_r",
    annot=True, fmt=".0f", linewidths=0.5,
    cbar=False,
    square=False,
    ax=ax
)

ax.set_title("Skin ADL vs H", loc='right', pad=10)  # pad=10 brings it closer
ax.set_ylabel("")
ax.set_xlabel("Rank")
ax.set_xticks([])  # remove x-axis tick labels
# ax.set_aspect(0.85)  # still control cell thickness

plt.tight_layout()
plt.savefig('../Plots/Analysis_figures/Random_Forest/top10_genera_rankings_all-classifications_skin-ADNL-H.png', dpi=600, bbox_inches='tight')





/var/folders/22/yck9vwx53w1c38tvj_c0_tz00000gn/T/ipykernel_47332/3902062598.py:29: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations.
  plt.tight_layout()


In [12]:
# --- Make sure Genus is the index ---
top10 = top10.reset_index().set_index('Genus')
top10.index = top10.index.str.strip()  # Clean any leading/trailing spaces

# Define which comparison
group4 = ["skin_ADNL_vs_ADL"]

# --- Select top 10 genera from skin_ADNL_vs_H only ---
top10_group4 = top10.sort_values('skin_ADNL_vs_ADL').head(10)

# --- Plot ---
fig, ax = plt.subplots(figsize=(1, 6))  # Slightly shorter figure

sns.heatmap(
    top10_group4[group4],
    cmap="Purples_r",
    annot=True, fmt=".0f", linewidths=0.5,
    cbar=False,
    square=False,
    ax=ax
)

ax.set_title("Skin ADL vs H", loc='right', pad=10)  # pad=10 brings it closer
ax.set_ylabel("")
ax.set_xlabel("Rank")
ax.set_xticks([])  # remove x-axis tick labels
# ax.set_aspect(0.85)  # still control cell thickness

plt.tight_layout()
plt.savefig('../Plots/Analysis_figures/Random_Forest/top10_genera_rankings_all-classifications_skin-ADNL-ADL.png', dpi=600, bbox_inches='tight')





/var/folders/22/yck9vwx53w1c38tvj_c0_tz00000gn/T/ipykernel_47332/4153687083.py:29: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations.
  plt.tight_layout()


In [13]:
# --- Make sure Genus is the index ---
top10 = top10.reset_index().set_index('Genus')
top10.index = top10.index.str.strip()  # Clean any leading/trailing spaces

# Define which comparison
group5 = ["nares_AD_vs_H"]

# --- Select top 10 genera from skin_ADL_vs_H only ---
top10_group5 = top10.sort_values('nares_AD_vs_H').head(10)

# --- Plot ---
fig, ax = plt.subplots(figsize=(1, 6))  # Slightly shorter figure

sns.heatmap(
    top10_group5[group5],
    cmap="Oranges_r",
    annot=True, fmt=".0f", linewidths=0.5,
    cbar=False,
    square=False,
    ax=ax
)

ax.set_title("Skin ADL vs H", loc='right', pad=10)  # pad=10 brings it closer
ax.set_ylabel("")
ax.set_xlabel("Rank")
ax.set_xticks([])  # remove x-axis tick labels
# ax.set_aspect(0.85)  # still control cell thickness

plt.tight_layout()
plt.savefig('../Plots/Analysis_figures/Random_Forest/top10_genera_rankings_all-classifications_nares-AD-H.png', dpi=600, bbox_inches='tight')





/var/folders/22/yck9vwx53w1c38tvj_c0_tz00000gn/T/ipykernel_47332/1709644833.py:29: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations.
  plt.tight_layout()
